In [1]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import pandas as pd
import json

In [2]:
with open("../to_delete.txt", "r") as fo:
    api_key = fo.read()

In [3]:
client = MistralClient(api_key=api_key)

In [4]:
chats = []

In [5]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt1 = f"""
Given the following data, what is the payment status for \
 transaction_id={transaction_id}?

data:
{data}

"""

prompt2 = f"""
Given the following data, what is the payment date for \
 transaction_id={transaction_id}?

data:
{data}

"""

In [6]:
def chat_completion_helper(content_msg, model="open-mistral-7b"):
    message = [ChatMessage(role="user", content=content_msg)]
    response = client.chat(model=model, messages=message)
    
    return response.choices[0].message.content

In [7]:
chat_completion_helper(prompt1)

'To find the payment status for transaction_id T1001, we can look at the "payment_status" key in the given data. Here, the payment status for transaction_id T1001 is "Paid".'

In [8]:
chat_completion_helper(prompt2)

'The payment date for transaction_id T1001 is "2021-10-05".'

## 2nd Section

In [9]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [10]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [11]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

{"status": "Paid"}


In [12]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [13]:
def retrieve_payment_amount(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"amount": df[df.transaction_id == transaction_id].payment_amount.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [14]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

{"date": "2021-10-06"}


In [15]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [16]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [17]:
tool_payment_amount = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_amount",
        "description": "Get payment amount of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [18]:
import functools
names_to_function = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
    "retrieve_payment_amount": functools.partial(retrieve_payment_amount, df=df),
}

In [19]:
names_to_function["retrieve_payment_amount"](transaction_id="T1001")

'{"amount": 125.5}'

In [20]:
tools = [tool_payment_status, tool_payment_date, tool_payment_amount]

In [21]:
tools

[{'type': 'function',
  'function': {'name': 'retrieve_payment_status',
   'description': 'Get payment status of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_date',
   'description': 'Get payment date of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}},
 {'type': 'function',
  'function': {'name': 'retrieve_payment_amount',
   'description': 'Get payment amount of a transaction',
   'parameters': {'type': 'object',
    'properties': {'transaction_id': {'type': 'string',
      'description': 'The transaction id.'}},
    'required': ['transaction_id']}}}]

In [33]:
chats = []
def chat_completion_helper(chats, model="mistral-large-latest"):
    print(chats, "\n\n\n\n")
    response = client.chat(model=model, messages=chats, tools=tools, tool_choice="auto")
    chats.append(response.choices[0].message)
    print(chats[-1], "\n\n\n\n")
    return chats

In [34]:
chats.append(ChatMessage(role="user", content=input("Enter input text?: ")))
chats = chat_completion_helper(chats=chats)

Enter input text?:  What is the status of my transaction


[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None)] 




role='assistant' content='To get the status of your transaction, I need the transaction ID. Could you please provide it?' name=None tool_calls=None 






In [35]:
chats.append(ChatMessage(role="user", content=input("Enter input text?: ")))
chats = chat_completion_helper(chats=chats)

Enter input text?:  My transaction_id is T1001


[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None)] 




role='assistant' content='' name=None tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))] 






In [26]:
function_name, json.loads(function_arg)

('retrieve_payment_status', {'transaction_id': 'T1001'})

In [27]:
names_to_function[function_name](**json.loads(function_arg))

'{"status": "Paid"}'

In [36]:
function_name = chats[-1].tool_calls[0].function.name
function_arg = chats[-1].tool_calls[0].function.arguments
user_execution_result = names_to_function[function_name](**json.loads(function_arg))
chats.append(ChatMessage(role="tool", content=user_execution_result))
chats = chat_completion_helper(chats=chats)

[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None)] 




role='assistant' content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?" name=None tool_calls=None 






## Exercise2

In [37]:
chats.append(ChatMessage(role="user", content=input("Enter input text?: ")))
chats = chat_completion_helper(chats=chats)

Enter input text?:  How much did I pay my recent order


[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None), ChatMessage(role='assistant', content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?", name=None, tool_calls=None), ChatMessage(role='user', content='How much did I pay my recent order', name=None, tool_calls=None)] 




role='assistant' content='To get the payment amount of 

In [38]:
chats.append(ChatMessage(role="user", content=input("Enter input text?: ")))
chats = chat_completion_helper(chats=chats)

Enter input text?:  Its the same transaction id as above


[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None), ChatMessage(role='assistant', content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?", name=None, tool_calls=None), ChatMessage(role='user', content='How much did I pay my recent order', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the payment am

In [39]:
function_name = chats[-1].tool_calls[0].function.name
function_arg = chats[-1].tool_calls[0].function.arguments
user_execution_result = names_to_function[function_name](**json.loads(function_arg))
chats.append(ChatMessage(role="tool", content=user_execution_result))
chats = chat_completion_helper(chats=chats)

[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None), ChatMessage(role='assistant', content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?", name=None, tool_calls=None), ChatMessage(role='user', content='How much did I pay my recent order', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the payment am

In [40]:
chats.append(ChatMessage(role="user", content=input("Enter input text?: ")))
chats = chat_completion_helper(chats=chats)

Enter input text?:  Can you also let me know the amount paid by transaction id T1002


[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None), ChatMessage(role='assistant', content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?", name=None, tool_calls=None), ChatMessage(role='user', content='How much did I pay my recent order', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the payment am

In [41]:
function_name = chats[-1].tool_calls[0].function.name
function_arg = chats[-1].tool_calls[0].function.arguments
user_execution_result = names_to_function[function_name](**json.loads(function_arg))
chats.append(ChatMessage(role="tool", content=user_execution_result))
chats = chat_completion_helper(chats=chats)

[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None), ChatMessage(role='assistant', content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?", name=None, tool_calls=None), ChatMessage(role='user', content='How much did I pay my recent order', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the payment am

In [42]:
chats.append(ChatMessage(role="user", content=input("Enter input text?: ")))
chats = chat_completion_helper(chats=chats)

Enter input text?:  No, thats it.. Thanks


[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]), ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None), ChatMessage(role='assistant', content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?", name=None, tool_calls=None), ChatMessage(role='user', content='How much did I pay my recent order', name=None, tool_calls=None), ChatMessage(role='assistant', content='To get the payment am

In [43]:
chats

[ChatMessage(role='user', content='What is the status of my transaction', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction_id is T1001', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ffnCjFjTQ', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]),
 ChatMessage(role='tool', content='{"status": "Paid"}', name=None, tool_calls=None),
 ChatMessage(role='assistant', content="The status of your transaction with ID T1001 is 'Paid'. Is there anything else you need help with?", name=None, tool_calls=None),
 ChatMessage(role='user', content='How much did I pay my recent order', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='To get the pay